In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import datajoint as dj
import h5py
import numpy as np

In [3]:
stimulus = dj.create_virtual_module('stimulus','pipeline_stimulus')

Connecting donnie@10.28.0.34:3306


In [4]:
target_scans = [{'animal_id': 22845, 'session': 7, 'scan_idx': 5},
                {'animal_id': 22845, 'session': 10, 'scan_idx': 5},
                {'animal_id': 22086, 'session': 2, 'scan_idx': 3},
                {'animal_id': 22904, 'session': 4, 'scan_idx': 2}]
data_file_names = ["fake_responses{}-{}-{}.h5".format(*target.values()) for target in target_scans]

In [5]:
from neuro_data.static_images.data_schemas import InputResponse, StaticMultiDatasetGroupAssignment

# real responses

In [6]:
# Run this block to generate h5 files

# for scan in target_scans:
    
#     gid = (StaticMultiDatasetGroupAssignment() & scan & 'description NOT like "alias%%"').fetch1('group_id')
    
#     target_name = "gid_{}_{}_{}_{}_response.h5".format(gid, scan['animal_id'],scan['session'],scan['scan_idx'])
#     responses = (InputResponse.ResponseBlock & scan).fetch1('responses')
    
#     with h5py.File(target_name) as fh:
#         fh['response'] = responses
        
    

In [16]:
# f = h5py.File('gid_81_22904_4_2_response.h5')

In [24]:
# f['response'].value.shape

(6000, 903)

# real responses rankings

In [7]:
from staticnet_mei_generation.MEIGenerationResult import UnitRanking
# UnitRanking.Unit & 'group_id = 78'
gid_table = StaticMultiDatasetGroupAssignment() & target_scans & 'description NOT like "alias%%"'

/usr/local/lib/python3.6/dist-packages/staticnet_analyses/multi_mei.py:1651: UserWarning: Use of this table is deprecated. It is kept only for record keeping purpose
  warnings.warn('Use of this table is deprecated. It is kept only for record keeping purpose')


In [8]:
table_keys = gid_table.fetch('KEY')

In [9]:
# original table model configuration
from staticnet_experiments.configs import NetworkConfig, TrainConfig, ReadoutConfig, ShifterConfig, DataConfig, CoreConfig, ModulatorConfig
from staticnet_experiments.models import Model
neurodata_static = dj.create_virtual_module('neurodata_static', 'neurodata_static')

# CoreConfig.GaussianLaplace Params
gauss_bias = 0.0
gauss_sigma = 0.5
input_kern = 15
hidden_kern = 7

core_restriction = dict(gauss_bias = 0.0, gauss_sigma = 0.5, input_kern = 15, hidden_kern = 7)

# configs TrainConfig.Default
batch_size = 60

# ReadoutConfig.SpatialTransformerPyramid2d() restrictions
downsample = False

# DataConfig.CorrectedAreaLayer() restriction
stimulus_type = 'stimulus.ColorFrameProjector'
exclude = ''
layer = 'L2/3'
brain_area = 'V1'
normalize_per_image = 'False'


# LINEAR MODEL
linear_CoreConfig_hash = (CoreConfig.StackedLinearGaussianLaplace() & 
                           dict(gauss_bias = gauss_bias,
                                gauss_sigma = gauss_sigma, 
                                input_kern = input_kern, 
                                hidden_kern = hidden_kern))

# NON-LINEAR MODEL
nonlinear_CoreConfig_hash = (CoreConfig.GaussianLaplace() & 
                           dict(gauss_bias = gauss_bias,
                                gauss_sigma = gauss_sigma, 
                                input_kern = input_kern, 
                                hidden_kern = hidden_kern))

TrainConfig_hash = (TrainConfig.Default & dict(batch_size = batch_size))

ReadoutConfig_hash = (ReadoutConfig.SpatialTransformerPyramid2d() & dict(downsample = downsample)) 

DataConfig_hash = (DataConfig.CorrectedAreaLayer() & 
                   dict(stimulus_type = stimulus_type, 
                        exclude = exclude, 
                        layer = layer, 
                        brain_area = brain_area, 
                        normalize_per_image=normalize_per_image))

nonlinear_model_config_restrictions = (NetworkConfig.CorePlusReadout() 
                    & nonlinear_CoreConfig_hash
                    & TrainConfig_hash
                    & ReadoutConfig_hash
                    & ShifterConfig.MLP() 
                    & ModulatorConfig.MLP() 
                    & DataConfig_hash)

linear_model_config_restrictions = (NetworkConfig.CorePlusReadout() 
                    & linear_CoreConfig_hash
                    & TrainConfig_hash
                    & ReadoutConfig_hash
                    & ShifterConfig.MLP() 
                    & ModulatorConfig.MLP() 
                    & DataConfig_hash)


In [10]:
UnitRanking & 'group_id = 79' & nonlinear_model_config_restrictions

group_id index of group,data_hash unique identifier for configuration,net_hash unique identifier for configuration,seed random seed,ranking_params
79,e7859d30ad45432f8dcfb0318d97a1a5,6c575960f9a52aefd333a7ac6826fff2,1009,1
79,e7859d30ad45432f8dcfb0318d97a1a5,706fd4edf1fe3409c71073c80b11e709,1009,1
79,e7859d30ad45432f8dcfb0318d97a1a5,8027407ca20cab25cd652fcca14135b8,1009,1
79,e7859d30ad45432f8dcfb0318d97a1a5,ab318018b9697dfc8f1e3b341221a381,1009,1


In [11]:

for t_key in table_keys:
    _hash = (Model & nonlinear_model_config_restrictions & 'seed=1009' & t_key).fetch('net_hash', order_by='val_corr DESC', limit=1)
    t_key['net_hash'] = _hash[0]

In [77]:
# # Run this block if you wanna make h5 files with real neuron response
# for t_key in table_keys:
#     neuron_id, rank = (UnitRanking.Unit & t_key).fetch('neuron_id', 'rank', order_by='neuron_id ASC')
#     file_name = 'gid_{}_{}_{}_{}_neuron_rank.h5'.format(t_key['group_id'],
#                                                         t_key['animal_id'],
#                                                         t_key['session'],
#                                                         t_key['scan_idx'])
#     with h5py.File(file_name) as fh:
#         fh['neuron_id'] = neuron_id
#         fh['rank'] = rank

In [16]:
Model & nonlinear_model_config_restrictions & 'seed=1009' & gid_table

group_id index of group,net_hash unique identifier for configuration,seed random seed,val_corr validation correlation (single trial),model stored model
78,8027407ca20cab25cd652fcca14135b8,1009,0.276089,=BLOB=
79,8027407ca20cab25cd652fcca14135b8,1009,0.328146,=BLOB=
80,8027407ca20cab25cd652fcca14135b8,1009,0.300268,=BLOB=
81,8027407ca20cab25cd652fcca14135b8,1009,0.269357,=BLOB=
78,6c575960f9a52aefd333a7ac6826fff2,1009,0.28839,=BLOB=
79,6c575960f9a52aefd333a7ac6826fff2,1009,0.346039,=BLOB=
80,6c575960f9a52aefd333a7ac6826fff2,1009,0.314403,=BLOB=
81,6c575960f9a52aefd333a7ac6826fff2,1009,0.283385,=BLOB=
78,ab318018b9697dfc8f1e3b341221a381,1009,0.271715,=BLOB=
79,ab318018b9697dfc8f1e3b341221a381,1009,0.327319,=BLOB=


In [17]:
nonlinear_model_config_restrictions

net_hash unique identifier for configuration,core_hash unique identifier for configuration,ro_hash unique identifier for configuration,shift_hash unique identifier for configuration,mod_hash unique identifier for configuration,train_hash unique identifier for configuration,data_hash unique identifier for configuration
8027407ca20cab25cd652fcca14135b8,00cfe341ff27a14a23c4e341eb40f67f,a206f6da6a16ea14081062a1e2436b48,05c69a4aeaeea5e48fa8fc5e70181d67,a757e992ae449e3057ff1d512a51bd1e,2c25349fa0a81afbe907882e7b50a4da,e7859d30ad45432f8dcfb0318d97a1a5
6c575960f9a52aefd333a7ac6826fff2,28bc2fa358337c5012278f899b5b6947,a206f6da6a16ea14081062a1e2436b48,05c69a4aeaeea5e48fa8fc5e70181d67,a757e992ae449e3057ff1d512a51bd1e,2c25349fa0a81afbe907882e7b50a4da,e7859d30ad45432f8dcfb0318d97a1a5
ab318018b9697dfc8f1e3b341221a381,7c7f81708088efc745206b49cc15723e,a206f6da6a16ea14081062a1e2436b48,05c69a4aeaeea5e48fa8fc5e70181d67,a757e992ae449e3057ff1d512a51bd1e,2c25349fa0a81afbe907882e7b50a4da,e7859d30ad45432f8dcfb0318d97a1a5
706fd4edf1fe3409c71073c80b11e709,86e5871c1e9112ad5c408697c2ad7d4d,a206f6da6a16ea14081062a1e2436b48,05c69a4aeaeea5e48fa8fc5e70181d67,a757e992ae449e3057ff1d512a51bd1e,2c25349fa0a81afbe907882e7b50a4da,e7859d30ad45432f8dcfb0318d97a1a5


# albums

In [7]:
imagenet = dj.create_virtual_module('imagenet','pipeline_imagenet')

In [8]:
collection1=((imagenet.Album.Single & 'image_class = "imagenet_v2_rgb"') &'collection_id = 1').fetch("image_id")
collection2=((imagenet.Album.Single & 'image_class = "imagenet_v2_rgb"') &'collection_id = 2').fetch("image_id")
collection3=((imagenet.Album.Single & 'image_class = "imagenet_v2_rgb"') &'collection_id = 3').fetch("image_id")

In [9]:
(imagenet.Album.Single & 'image_class = "imagenet_v2_rgb"') &'collection_id = 1'

collection_id image collection id,image_class,image_id image id
1,imagenet_v2_rgb,17
1,imagenet_v2_rgb,23
1,imagenet_v2_rgb,36
1,imagenet_v2_rgb,62
1,imagenet_v2_rgb,96
1,imagenet_v2_rgb,117
1,imagenet_v2_rgb,121
1,imagenet_v2_rgb,157
1,imagenet_v2_rgb,165
1,imagenet_v2_rgb,181


In [18]:
for scan in target_scans:
    album_num = 5
    name = '{}_{}_{}_album_{}.h5'.format(*scan.values(),album_num)
    print(name)

22845_7_5_album_5.h5
22845_10_5_album_5.h5
22086_2_3_album_5.h5
22904_4_2_album_5.h5


In [40]:

# # Run this block to save album images into h5 files

# for scan in target_scans:
#     print(scan)
#     condition = (stimulus.ColorFrameProjector & (stimulus.Condition & (stimulus.Trial & scan)))
    
#     for album_num in range(1,4):
        
#         single = imagenet.Album.Single & 'collection_id = {} AND image_class = "imagenet_v2_rgb"'.format(album_num)
#         oracle = imagenet.Album.Oracle & 'collection_id = {} AND image_class = "imagenet_v2_rgb"'.format(album_num)
    
#         print('For album number {}, there are {} unique images found'.format(album_num, len((single + oracle) & condition)))
        
#         if len((single + oracle) & condition) >0:
            
#             images = ((stimulus.Trial & scan) * condition * stimulus.StaticImage.Image).fetch('image')
            
#             file_name = '{}_{}_{}_album_num_{}.h5'.format(*scan.values(),album_num)
            
#             with h5py.File(file_name) as fh:
#                 fh['album'] = np.stack(images)

{'animal_id': 22845, 'session': 7, 'scan_idx': 5}
For album number 1, there are 5100 unique images found
For album number 2, there are 0 unique images found
For album number 3, there are 0 unique images found
{'animal_id': 22845, 'session': 10, 'scan_idx': 5}
For album number 1, there are 5100 unique images found
For album number 2, there are 0 unique images found
For album number 3, there are 0 unique images found
{'animal_id': 22086, 'session': 2, 'scan_idx': 3}
For album number 1, there are 0 unique images found
For album number 2, there are 5100 unique images found
For album number 3, there are 0 unique images found
{'animal_id': 22904, 'session': 4, 'scan_idx': 2}
For album number 1, there are 0 unique images found
For album number 2, there are 0 unique images found
For album number 3, there are 5100 unique images found


In [32]:
(stimulus.Trial & scan) * condition * stimulus.StaticImage.Image

animal_id id number,session session index for the mouse,scan_idx number of TIFF stack file,trial_idx trial index within sessions,condition_hash 120-bit hash (The first 20 chars of MD5 in base64),image_class,image_id image id,flip_times (s) row array of flip times,last_flip the last flip number in this trial,trial_ts automatic,projector_id projector id,projector_config_id projector config,channel_1 which color channel of the image is mapped to channel 1 of the projector,channel_2 which color channel of the image is mapped to channel 2 of the projector,channel_3 which color channel of the image is mapped to channel 3 of the projector,pre_blank_period (s) off duration,presentation_time (s) image duration,image actual image
22904,4,2,0,NRXzk64i/JFfvvQ2O8/t,imagenet_v2_rgb,28564,=BLOB=,1604,2019-12-09 14:31:43,1,6,2,3,None,0.470163,0.5,=BLOB=
22904,4,2,1,h88vV7vciIP10g8CWVJS,imagenet_v2_rgb,44352,=BLOB=,1607,2019-12-09 14:31:45,1,6,2,3,None,0.499462,0.5,=BLOB=
22904,4,2,2,nOjfxj+drqkYGm14nwpx,imagenet_v2_rgb,100048,=BLOB=,1610,2019-12-09 14:31:45,1,6,2,3,None,0.311245,0.5,=BLOB=
22904,4,2,3,idaZSlnLD8mqDOStruQv,imagenet_v2_rgb,20751,=BLOB=,1613,2019-12-09 14:31:47,1,6,2,3,None,0.458631,0.5,=BLOB=
22904,4,2,4,P4hsAQz032nLaOpYVD9X,imagenet_v2_rgb,72157,=BLOB=,1616,2019-12-09 14:31:47,1,6,2,3,None,0.346544,0.5,=BLOB=
22904,4,2,5,lGdca13HIk37iMPeP1QG,imagenet_v2_rgb,66689,=BLOB=,1619,2019-12-09 14:31:48,1,6,2,3,None,0.302255,0.5,=BLOB=
22904,4,2,6,bmI3DAcwuV/D/jJzivKY,imagenet_v2_rgb,28314,=BLOB=,1622,2019-12-09 14:31:49,1,6,2,3,None,0.438943,0.5,=BLOB=
22904,4,2,7,0nq3Q9V6nzPdjTxNkDIb,imagenet_v2_rgb,35075,=BLOB=,1625,2019-12-09 14:31:50,1,6,2,3,None,0.401733,0.5,=BLOB=
22904,4,2,8,WZy1U/6QRW3KyQ/OD3wi,imagenet_v2_rgb,53777,=BLOB=,1628,2019-12-09 14:31:51,1,6,2,3,None,0.385587,0.5,=BLOB=
22904,4,2,9,hGg9mHEp5lxW1Vx89y7y,imagenet_v2_rgb,16314,=BLOB=,1631,2019-12-09 14:31:52,1,6,2,3,None,0.489965,0.5,=BLOB=


In [16]:
(single + oracle) & (stimulus.ColorFrameProjector & (stimulus.Condition & (stimulus.Trial & target_scans[0])))

collection_id image collection id,image_class,image_id image id
1,imagenet_v2_rgb,17
1,imagenet_v2_rgb,23
1,imagenet_v2_rgb,36
1,imagenet_v2_rgb,62
1,imagenet_v2_rgb,96
1,imagenet_v2_rgb,117
1,imagenet_v2_rgb,121
1,imagenet_v2_rgb,157
1,imagenet_v2_rgb,165
1,imagenet_v2_rgb,181


In [10]:
conditions6000 = (stimulus.Trial & target_scans[0]) & (stimulus.ColorFrameProjector & (stimulus.Condition & (stimulus.Trial & target_scans[0])))

In [69]:
conditions6000 * stimulus.ColorFrameProjector * stimulus.StaticImage.Image

animal_id id number,session session index for the mouse,scan_idx number of TIFF stack file,trial_idx trial index within sessions,condition_hash 120-bit hash (The first 20 chars of MD5 in base64),image_class,image_id image id,flip_times (s) row array of flip times,last_flip the last flip number in this trial,trial_ts automatic,projector_id projector id,projector_config_id projector config,channel_1 which color channel of the image is mapped to channel 1 of the projector,channel_2 which color channel of the image is mapped to channel 2 of the projector,channel_3 which color channel of the image is mapped to channel 3 of the projector,pre_blank_period (s) off duration,presentation_time (s) image duration,image actual image
22845,7,5,0,yK+us+j2TPPBxu9H5Vyg,imagenet_v2_rgb,34248,=BLOB=,4271,2019-11-20 14:02:15,1,6,2,3,None,0.48624,0.5,=BLOB=
22845,7,5,1,4dZOSuKQif5u/V6vLkWQ,imagenet_v2_rgb,109205,=BLOB=,4274,2019-11-20 14:02:16,1,6,2,3,None,0.308308,0.5,=BLOB=
22845,7,5,2,BILdQS6dITVn72l3dVHW,imagenet_v2_rgb,25699,=BLOB=,4277,2019-11-20 14:02:17,1,6,2,3,None,0.383798,0.5,=BLOB=
22845,7,5,3,J59yUdCWBNFjGi5+FbUD,imagenet_v2_rgb,37112,=BLOB=,4280,2019-11-20 14:02:18,1,6,2,3,None,0.344535,0.5,=BLOB=
22845,7,5,4,wZdRSXJlkcoQUHuvnmhL,imagenet_v2_rgb,101460,=BLOB=,4283,2019-11-20 14:02:19,1,6,2,3,None,0.35355,0.5,=BLOB=
22845,7,5,5,8iK+Xrb/ZoFFXq64dtUW,imagenet_v2_rgb,71509,=BLOB=,4286,2019-11-20 14:02:20,1,6,2,3,None,0.422571,0.5,=BLOB=
22845,7,5,6,tp+xEhKYD3RRzv0bDTnX,imagenet_v2_rgb,120408,=BLOB=,4289,2019-11-20 14:02:21,1,6,2,3,None,0.303113,0.5,=BLOB=
22845,7,5,7,b7a4MZVOkurYn7Ou+mJz,imagenet_v2_rgb,67302,=BLOB=,4292,2019-11-20 14:02:22,1,6,2,3,None,0.3203,0.5,=BLOB=
22845,7,5,8,H+vGS6sF1EZqzuMKnkoK,imagenet_v2_rgb,58119,=BLOB=,4295,2019-11-20 14:02:23,1,6,2,3,None,0.406858,0.5,=BLOB=
22845,7,5,9,l/d41+xGIpMqSWZmalq4,imagenet_v2_rgb,93051,=BLOB=,4298,2019-11-20 14:02:24,1,6,2,3,None,0.495604,0.5,=BLOB=
